In [2]:
import csv
import os
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import time
from pathlib import Path

In [3]:
selected_refs = np.load('selected_datasets_ref.npy')

In [26]:
len(selected_refs)

4052

In [27]:
folders = [ref.replace('/', '_') for ref in selected_refs]
folders[0]

'rabemampiandraeric_currency'

# Cleaning and Checking Downloaded Datasets

In [ ]:
%cd datasets

Find refs of missing datasets.

In [28]:
missing_folders = []
for folder in folders:
    if not os.path.isdir(folder):
        missing_folders.append(folder)
print('Missing %d datasets: %s' % (len(missing_folders), missing_folders))

Missing 7 datasets: ['ajaxfb_prodemg-services', 'jackvial_themaestrodatasetv2', 'sandhaya4u_histology-image-dataset', 'williamkaiser_all-isef-projects', 'roydatascience_chemistry-models', 'wjhyde1_2014-acs-dashboard', 'artemnaida_canadian-economic-data']


Remove refs of missing datasets.

In [29]:
size_before = len(folders)
for missing_folder in missing_folders:
    folders.remove(missing_folder)
print('Remove %d folder names' % (size_before - len(folders)))

Remove 7 folder names


Check if all datasets folders contain only files or have nested directories.

In [30]:
folder_with_nonfiles = []
for folder in folders:
    folder_items = os.listdir(folder)
    for item in folder_items:
        if not os.path.isfile(os.path.join(folder, item)):
            folder_with_nonfiles.append(folder)
            break
print('Found %d folders with nonfiles (directories)' % len(folder_with_nonfiles))

Found 136 folders with nonfiles (directories)


In [35]:
for folder in folder_with_nonfiles:
    items = os.listdir(folder)
    sub_folders = [item for item in items if os.path.isdir(os.path.join(folder, item))]
    for sub_folder in sub_folders:
        items = os.listdir(os.path.join(folder, sub_folder))
        sub_sub_folders = [item for item in items if os.path.isdir(os.path.join(folder, sub_folder, item))]
        if len(sub_sub_folders) > 0:
            print('\n' + folder)
            print(sub_sub_folders)


hunhthanhphong_health-news-in-twitter-uci
['Health-Tweets']

hariwh0_vietnam-stock-competition
['modules']

ayushranjan1610_movies
['.ipynb_checkpoints']

husnantaj_review-scrapper
['.idea', 'static', 'templates']

dataup1_ogbg-moltox21
['scaffold']

sgreiner_arc-rasp
['output']

vin1234_detecting-sentiments-of-a-quote
['Dataset', 'Sample Data Files']

evanregan_dsci-511-us-county-health-dataset
['pickles']

sripaadsrinivasan_prokabadiseassion17stats
['Data']

xenophule_udemy-class-pca
['PCA']

rafaelgreca_wnba-games-box-score-since-1997
['Data']

brettlines_bbc-scotland-house-party-data
['paul']

ryuuseikuhome_azuralane-ships-data
['blhx_avatar']

tabarkarajab_decktrot-cards-dataset
['test dataset', 'train dataset']

bangdaeho_korea-money
['images', 'labels']

teguhpermana_data-analysis-gdp-per-capita-usd
['latihan1']

teguhpermana_data-analysis-gdp-per-capita-usd
['lifesat']

teguhpermana_data-analysis-gdp-per-capita-usd
['fundamentals']

irwansyah10010_mobilelegend
['Atribut', 'Ski

Walk the dataset directories in depth. For each dataset, we collect the name of all files into a dict.

In [167]:
def get_flat_list_files(folder_to_file_names: dict[str, list]) -> list:
    return [val for sublist in list(folder_to_file_names.values()) for val in sublist]

In [168]:
folder_to_file_names = {}
for folder in folders:
    folder_to_file_names[folder] = []
    for path, subdirs, files in os.walk(folder):
        for name in files:
            folder_to_file_names[folder].append(os.path.join(path, name))
print('%d folders total, %d files total' % (len(folder_to_file_names), len(get_flat_list_files(folder_to_file_names))))

4045 folders total, 200817 files total


Check how may file types we have.

In [169]:
file_types = set()
for folder, files in folder_to_file_names.items():
    for file_path_str in files:
        type = Path(file_path_str).suffix
        file_types.add(type)
print('%d file types' % len(file_types))  
print(file_types)  

90 file types
{'', '.JPG', '.mat', '.pdf', '.model', '.hdf5', '.ffn', '.CSV', '.db', '.whl', '.jpg', '.R', '.pth', '.dat', '.xz', '.index', '.json', '.patch', '.PNG', '.bgz', '.sqlite', '.mod', '.names', '.shp', '.nex', '.fasta', '.txt', '.waypoints', '.Rtab', '.seq', '.pptx', '.xlsx', '.faa', '.iml', '.log', '.parquet', '.shx', '.geojson', '.css', '.sql', '.gb', '.fna', '.jfif', '.lua', '.prj', '.dot', '.yml', '.sav', '.in', '.fa', '.win', '.aln', '.Rd', '.sh', '.rtf', '.npy', '.go', '.arff', '.docx', '.cfg', '.py', '.a3m', '.js', '.c', '.xml', '.html', '.dbf', '.sum', '.pgn', '.yxmd', '.content', '.jpeg', '.pt', '.png', '.pickle', '.tsv', '.csv', '.sample', '.gbk', '.pkl', '.data-00000-of-00001', '.pdb', '.gbwithparts', '.emb', '.mp3', '.md', '.rds', '.gp', '.xls', '.ipynb'}


Remove folders that only have models (e.g., remove husnantaj_review-scrapper as it has no data, it's just a scraper).

In [170]:
folder_to_file_names.pop('husnantaj_review-scrapper')
print('%d folders total' % len(folder_to_file_names))

4044 folders total


Also, change 'paruqet' to 'parquet' manually (mispelling) .

Remove non-data file extensions.

In [279]:
# *Note*: if folder contains '.md' files most likely it is a README, Contributing, or a model file.
types_to_remove = ['.pptx', '.lua', '.patch', '.iml', '.yml', '.ipynb', '.R', '.pdf', '.c', '.js', '.py', '.go']

for folder, files in folder_to_file_names.items():
    files_to_remove_from_folder = []
    for file_path_str in files:
        type = Path(file_path_str).suffix
        if type in types_to_remove:
            files_to_remove_from_folder.append(file_path_str)
    if len(files_to_remove_from_folder) > 0:
        print("Removing from folder %s the following files: %s" % (folder, files_to_remove_from_folder))
        for file_to_remove in files_to_remove_from_folder:
            folder_to_file_names[folder].remove(file_to_remove)
print('%d folders total' % len(get_flat_list_files(folder_to_file_names)))

200357 folders total


Filter out any folders (datasets) that have no files left after above removal.

In [188]:
for folder, files in folder_to_file_names.items():
    if len(files) == 0:
        print('Removing empty folder %s' % folder)
        folder_to_file_names.pop(folder)
print('%d folders total' % len(folder_to_file_names))

4044 folders total


# Feature Extraction (quality attributes)

In [177]:
# Remaining file types
remaining_file_types = set()
for folder, files in folder_to_file_names.items():
    for file_path_str in files:
        type = Path(file_path_str).suffix
        remaining_file_types.add(type)
print('%d file types remaining: %s' % (len(remaining_file_types), str(remaining_file_types)))

79 file types remaining: {'', '.JPG', '.mat', '.model', '.hdf5', '.xls', '.ffn', '.CSV', '.db', '.whl', '.jpg', '.pth', '.dat', '.xz', '.index', '.json', '.PNG', '.bgz', '.sqlite', '.mod', '.names', '.shp', '.nex', '.fasta', '.txt', '.waypoints', '.seq', '.xlsx', '.faa', '.log', '.parquet', '.shx', '.geojson', '.css', '.sql', '.gb', '.fna', '.jfif', '.prj', '.dot', '.sav', '.in', '.fa', '.win', '.aln', '.Rd', '.sh', '.rtf', '.npy', '.go', '.arff', '.docx', '.cfg', '.a3m', '.xml', '.html', '.dbf', '.sum', '.pgn', '.yxmd', '.content', '.jpeg', '.pt', '.png', '.pickle', '.tsv', '.csv', '.sample', '.gbk', '.pkl', '.data-00000-of-00001', '.pdb', '.gbwithparts', '.emb', '.mp3', '.md', '.rds', '.gp', '.Rtab'}


Let's work only with csv and parquet for now.

In [ ]:
filtered_types = ['.csv'] #'.xls', '.xlsx', '.json', '.geojson', '.CSV', '.parquet' 

Profiler works for '.csv', '.parquet', '.avro', '.json', graph, and text files.

In [178]:
folder_to_file_names['sourabhji_availabilty-of-groundwater-future-use']

['sourabhji_availabilty-of-groundwater-future-use\\Groundwater.csv',
 'sourabhji_availabilty-of-groundwater-future-use\\mytree.dot']

In [ ]:
import json
from dataprofiler import Data, Profiler

data = Data("sourabhji_availabilty-of-groundwater-future-use\\Groundwater.csv") # Auto-Detect & Load: CSV, AVRO, Parquet, JSON, Text

profile = Profiler(data) # Calculate Statistics, Entity Recognition, etc
report = profile.report(report_options={"output_format":"pretty"})
print(json.dumps(report, indent=4))

In [250]:
global_stats = report['global_stats']
data_stats = report['data_stats']

### Consistency

- Composition

In [271]:
# Num of integer features (int)
n_int_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] == 'integer':
        n_int_features += 1

# Num of float features (int)
n_float_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] == 'float':
        n_float_features += 1

# Num of string features (int)
n_string_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] == 'string':
        n_string_features += 1

# Num of datetime features (int)
n_datetime_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] == 'datetime':
        n_datetime_features += 1

# Num of categorical features (int)
n_categorical_features = 0
for feature_stats in data_stats:
    if feature_stats['categorical'] is True:
        n_categorical_features += 1

# Are data types consistent throughout the dataset? (one-hot) TODO: do for numerical types as well
is_datetime_consistent = True
for feature_stats in data_stats:
    if feature_stats['data_type'] == 'datetime':
        if 'format' in feature_stats['statistics']:
            format_list_str = feature_stats['statistics']['format']
            n_datetime_formats = len(format_list_str.split(','))
            if format_list_str == '[]' or n_datetime_formats > 1:
                is_datetime_consistent = False
                break

# At least one feature has ordered values? (one-hot)
is_a_feature_ordered = True
for feature_stats in data_stats:
    if feature_stats['order'] != 'random':
        is_a_feature_ordered = True
        break

print('n_int_features=%d' % n_int_features)
print('n_float_features=%d' % n_float_features)
print('n_string_features=%d' % n_string_features)
print('n_datetime_features=%d' % n_datetime_features)
print('n_categorical_features=%d' % n_categorical_features)
print('is_datetime_consistent=%d' % is_datetime_consistent)
print('is_a_feature_ordered=%d' % is_a_feature_ordered)


n_int_features=0
n_float_features=12
n_string_features=2
n_datetime_features=0
n_categorical_features=1
n_categorical_features=1
is_datetime_consistent=1
is_a_feature_ordered=1


- File Format

In [252]:
# File format extension (type)? (e.g., csv, xlsx, txt, etc.)
# TODO


# Structure type? (i.e., structured, semi-structured, unstructured)
structure_to_extension = {
    'structured': set(['csv', 'parquet', 'xls', 'xlsx', 'hdf5', 'sqlite', 'sql', 'pickle', 'pgn', ]),
    'semi-structured': set(['xml', 'json', 'geojson', 'html', 'css']),
    # Everything else is unstructured
}
# TODO


- Duplication and Nullness

In [272]:
# Contains duplicated rows?
has_duplicated_rows = 0 if global_stats['duplicate_row_count'] == 0 else 1

# Contains null rows?
has_null_rows = 0 if global_stats['row_is_null_ratio'] == 0 else 1

# Contains null feature values?
has_null_feature_values = 0 if global_stats['row_has_null_ratio'] == 0 else 1

print('has_duplicated_rows=%d' % has_duplicated_rows)
print('has_null_rows=%d' % has_null_rows)
print('has_null_feature_values=%d' % has_null_feature_values)

has_duplicated_rows=0
has_null_rows=0
has_null_feature_values=0


### Completeness

- Volume and Size

In [254]:
# Num of features
n_features = global_stats['column_count']

# Num rows
n_rows = global_stats['column_count']

# Num rows x num features
n_rows_times_features = n_features * n_rows

# Num bytes
# TODO

print('n_features=%d' % n_features)
print('n_rows=%d' % n_rows)
print('n_rows_times_features=%d' % n_rows_times_features)

- Duplication and Nullness

In [275]:
# Average ratio of duplication across non-categorical features
avg_ratio_duplication_noncateg_features = 0
for feature_stats in data_stats:
    if feature_stats['categorical'] is False:
        avg_ratio_duplication_noncateg_features += feature_stats['statistics']['unique_ratio']
avg_ratio_duplication_noncateg_features /= n_categorical_features

# Ratio of features that contain at least one null value
ratio_features_at_least_one_null = global_stats['row_has_null_ratio']

# Average ratio of null values across features
avg_ratio_null_values_features = 0
for feature_stats in data_stats:
    avg_ratio_null_values_features += feature_stats['statistics']['null_count'] / n_rows
avg_ratio_null_values_features /= n_features

print('avg_ratio_duplication_noncateg_features=%f' % avg_ratio_duplication_noncateg_features)
print('ratio_features_at_least_one_null=%f' % ratio_features_at_least_one_null)
print('avg_ratio_null_values_features=%f' % avg_ratio_null_values_features)

avg_ratio_duplication_noncateg_features=10.862000
ratio_features_at_least_one_null=0.000000
avg_ratio_null_values_features=0.000000


- Statistical Properties

In [276]:
# # # # # 1. Numerical features
numerical_types = ['integer', 'float']
n_numerical_features = n_int_features + n_float_features
# Average variance across features for int, float (numerical) data types
avg_variance_numerical_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] in numerical_types:
        avg_variance_numerical_features += feature_stats['statistics']['variance']
avg_variance_numerical_features /= n_numerical_features

# Average skewness across features for int, float (numerical) data types
avg_skewness_numerical_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] in numerical_types:
        avg_skewness_numerical_features += feature_stats['statistics']['skewness']
avg_skewness_numerical_features /= n_numerical_features

# Average kurtosis across features for int, float (numerical) data types
avg_kurtosis_numerical_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] in numerical_types:
        avg_kurtosis_numerical_features += feature_stats['statistics']['kurtosis']
avg_kurtosis_numerical_features /= n_numerical_features

# Average median absolute deviation across features for int, float (numerical) data types
avg_median_dev_numerical_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] in numerical_types:
        avg_median_dev_numerical_features += feature_stats['statistics']['median_abs_deviation']
avg_median_dev_numerical_features /= n_numerical_features

# Average number of outliers across numerical features
avg_n_outliers_numerical_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] in numerical_types:
        q1 = feature_stats['statistics']['quantiles'][0]
        q2 = feature_stats['statistics']['quantiles'][1]
        q3 = feature_stats['statistics']['quantiles'][2]
        iqr = q3 - q1
        lav = q1 - 1.5 * iqr
        uav = q3 + 1.5 * iqr

        data_type = 'int64'
        if feature_stats['data_type'] == 'float':
            data_type = 'float64'
        
        try:
            df_column = data.data[feature_stats['column_name']].to_numpy(dtype=data_type, na_value=q2)
        except Exception as e:
            # TODO: exception message to include ref and column name
            print(e)
        avg_n_outliers_numerical_features += len(np.where((df_column > uav) & (df_column < lav)))
avg_n_outliers_numerical_features /= n_numerical_features

# # # # # 2. Categorical features
# Average unalikeability impurity across categorical features
avg_unalikeability_categ_features = 0
for feature_stats in data_stats:
    if feature_stats['categorical'] is True:
        avg_unalikeability_categ_features += feature_stats['statistics']['unalikeability']
avg_unalikeability_categ_features /= n_categorical_features


print('avg_variance_numerical_features=%f' % avg_variance_numerical_features)
print('avg_skewness_numerical_features=%f' % avg_skewness_numerical_features)
print('avg_kurtosis_numerical_features=%f' % avg_kurtosis_numerical_features)
print('avg_median_dev_numerical_features=%f' % avg_median_dev_numerical_features)
print('avg_n_outliers_numerical_features=%f' % avg_n_outliers_numerical_features)
print('avg_unalikeability_categ_features=%f' % avg_unalikeability_categ_features)

avg_variance_numerical_features=80.732275
avg_skewness_numerical_features=1.805717
avg_kurtosis_numerical_features=4.453533
avg_median_dev_numerical_features=3.152125
avg_n_outliers_numerical_features=1.000000
avg_unalikeability_categ_features=0.716700


### Accuracy

In [277]:
# Precision - average margin of error for numerical (int, float) features
avg_margin_error_numerical_features = 0
for feature_stats in data_stats:
    if feature_stats['data_type'] in numerical_types:
        avg_margin_error_numerical_features += feature_stats['statistics']['precision']['margin_of_error']
avg_margin_error_numerical_features /= n_numerical_features

# Average Gini impurity across categorical features
avg_gini_impurity_categ_features = 0
for feature_stats in data_stats:
    if feature_stats['categorical'] is True:
        avg_gini_impurity_categ_features += feature_stats['statistics']['gini_impurity']
avg_gini_impurity_categ_features /= n_categorical_features

print('avg_margin_error_numerical_features=%f' % avg_margin_error_numerical_features)
print('avg_gini_impurity_categ_features=%f' % avg_gini_impurity_categ_features)

avg_margin_error_numerical_features=0.801867
avg_gini_impurity_categ_features=0.692000


### Timeliness

In [ ]:
# Difference between current time and last update time
time_diff_current_and_last_update = time.time() - last_update_time

In [20]:
time.strptime('8/5/2020 11:57:10 AM', '%m/%d/%Y %H:%M:%S %p')

time.struct_time(tm_year=2020, tm_mon=8, tm_mday=5, tm_hour=11, tm_min=57, tm_sec=10, tm_wday=2, tm_yday=218, tm_isdst=-1)

In [5]:
d = {'a':1, 'b':2}
d.items()

dict_items([('a', 1), ('b', 2)])

### Privacy

In [278]:
# Aproximate num of PII entities
pii_labels = ['ADDRESS', 'CREDIT_CARD', 'DRIVERS_LICENSE', 'EMAIL_ADDRESS', 'UUID', 'IPV4', 'IPV6', 'MAC_ADDRESS', 'PERSON', 'PHONE_NUMBER', 'SSN']
num_pii_entities = 0
for feature_stats in data_stats:
    predicted_label_ratios = feature_stats['statistics']['data_label_representation']
    for pii_label in pii_labels:
        num_pii_entities += predicted_label_ratios[pii_label] * n_rows
num_pii_entities
print('num_pii_entities=%f' % num_pii_entities)

num_pii_entities=0.483000


# Improving performance of Data Profiler

In [113]:
x = 30
num_processes = 3
x * 1000 / 60 / 60 / (num_processes * 0.7) 

3.9682539682539693

In [127]:
import csv
import os
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import time
from pathlib import Path
import json
from dataprofiler import Data, Profiler, ProfilerOptions

In [132]:
# Small file: 30 rows, 14 cols
small_file = "datasets\\sourabhji_availabilty-of-groundwater-future-use\\Groundwater.csv"
# Medium file: 8100 rows, 20+ cols
medium_file = "datasets\\uciml_mushroom-classification\\mushrooms.csv"
# Large file: 340k rows, 8 cols
large_file = "datasets\\akhilv11_border-crossing-entry-data\\Border_Crossing_Entry_Data.csv"

# Load file (CSV should be automatically identified)
start = time.time()
data = Data(large_file)
end = time.time()
print('\nLoaded datasets in %.2f\n' % (end - start))


Loaded datasets in 0.32



In [134]:

# Profilers options (disable unused features to reduce compute time)
profile_options = ProfilerOptions()
# # Structured options
profile_options.structured_options.multiprocess.is_enabled = False
profile_options.structured_options.chi2_homogeneity.is_enabled = False
# # # Numerical (int, float)
# # # # Integer
profile_options.structured_options.int.min.is_enabled = False
profile_options.structured_options.int.max.is_enabled = False
profile_options.structured_options.int.num_zeros.is_enabled = False
profile_options.structured_options.int.num_negatives.is_enabled = False
# # # # Float
profile_options.structured_options.float.min.is_enabled = False
profile_options.structured_options.float.max.is_enabled = False
profile_options.structured_options.float.num_zeros.is_enabled = False
profile_options.structured_options.float.num_negatives.is_enabled = False
# # # Text
profile_options.structured_options.text.vocab.is_enabled = False
profile_options.structured_options.text.min.is_enabled = False
profile_options.structured_options.text.max.is_enabled = False
profile_options.structured_options.text.mode.is_enabled = False
profile_options.structured_options.text.median.is_enabled = False
profile_options.structured_options.text.sum.is_enabled = False
profile_options.structured_options.texta.variance.is_enabled = False
profile_options.structured_options.text.skewness.is_enabled = False
profile_options.structured_options.text.kurtosis.is_enabled = False
profile_options.structured_options.text.histogram_and_quantiles.is_enabled = False
profile_options.structured_options.text.median_abs_deviation.is_enabled = False


start = time.time()
# Profile the dataset
print(data.data.shape[0])
profile = Profiler(data, samples_per_update=data.data.shape[0], options=profile_options)
end = time.time()
print('\nProfiler took %.2f\n' % (end - start))


# Print the report
# print(json.dumps(report, indent=4))

346733


C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profiler_options.py:529: UserWarning: StructuredOptions.text.numeric_stats: The numeric stats are completely disabled.
  warnings.warn(


INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


100%|██████████| 8/8 [00:04<00:00,  1.67it/s]

INFO:DataProfiler.profilers.profile_builder: Calculating the statistics... 



100%|██████████| 8/8 [00:17<00:00,  2.19s/it]



Profiler took 28.10



In [77]:
# Generate a report and use json to prettify.
start = time.time()
report  = profile.report(report_options={"output_format": "serializable"})
end = time.time()
print('Report took %.2f\n' % (end - start))

Report took 0.06



In [119]:
selected_refs = np.load('selected_datasets_ref1.npy')

In [3]:
df = pl.read_csv('selected_datasets_for_download_with_proxy.csv')

In [5]:
df['ref'].to_numpy()

array(['dipam7/student-grade-prediction',
       'akhilv11/border-crossing-entry-data',
       'floser/french-motor-claims-datasets-fremtpl2freq', ...,
       'sandeep2812/covid19casestudyjohnshopkinsuniversitydaily',
       'sampledemoproject/covid-study-impact-data',
       'manjarinandimajumdar/essayscsv'], dtype=object)

In [6]:
pl.from_dicts([{'a': 1, 'b': 2}, {'a': 5, 'b': 7}])

a,b
i64,i64
1,2
5,7


In [2]:
[1, 2, 3][1:2]

[2]